### 1. Import data

In [1]:
from mnist.loader import MNIST

In [2]:
data = MNIST(path='data/', return_type='numpy')
data.select_emnist('letters')
X, y = data.load_training()

In [3]:
print(X.shape, y.shape)

(124800, 784) (124800,)


In [4]:
X = X.reshape(124800, 28, 28)
y = y.reshape(124800, 1)

In [5]:
y=y-1

### 2. Train-Test split

In [6]:
# pip install scikit-learn
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=50)

In [8]:
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

In [9]:
# pip install tensorflow
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes = 26)
y_test = to_categorical(y_test, num_classes = 26)

### 3. Define our model

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
 
model = Sequential()
model.add(Flatten(input_shape = (28,28)))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2)) # preventing overfitting
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(26, activation='softmax'))
 

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 26)                13338     
                                                                 
Total params: 677914 (2.59 MB)
Trainable params: 677914 

In [12]:
model.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

### 4. Calculate Accuracy

In [13]:
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]
print("Before training, test accuracy is", accuracy)



Before training, test accuracy is 3.537660092115402


In [14]:

from keras.callbacks import ModelCheckpoint
 
checkpointer = ModelCheckpoint(filepath = 'best_model.h5', verbose=1, save_best_only = True)
model.fit(X_train, y_train, batch_size = 128, epochs= 10, validation_split = 0.2, 
          callbacks=[checkpointer], verbose=1, shuffle=True)


Epoch 1/10
615/624 [============================>.] - ETA: 0s - loss: 0.8078 - accuracy: 0.7556
Epoch 1: val_loss improved from inf to 0.42855, saving model to best_model.h5
624/624 [==============================] - 5s 7ms/step - loss: 0.8038 - accuracy: 0.7567 - val_loss: 0.4285 - val_accuracy: 0.8667
Epoch 2/10
 24/624 [>.............................] - ETA: 4s - loss: 0.4361 - accuracy: 0.8568

C:\Users\vaibh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


618/624 [============================>.] - ETA: 0s - loss: 0.4227 - accuracy: 0.8651
Epoch 2: val_loss improved from 0.42855 to 0.34937, saving model to best_model.h5
624/624 [==============================] - 4s 7ms/step - loss: 0.4222 - accuracy: 0.8651 - val_loss: 0.3494 - val_accuracy: 0.8883
Epoch 3/10
623/624 [============================>.] - ETA: 0s - loss: 0.3417 - accuracy: 0.8886
Epoch 3: val_loss improved from 0.34937 to 0.31199, saving model to best_model.h5
624/624 [==============================] - 4s 7ms/step - loss: 0.3417 - accuracy: 0.8886 - val_loss: 0.3120 - val_accuracy: 0.9003
Epoch 4/10
617/624 [============================>.] - ETA: 0s - loss: 0.2959 - accuracy: 0.8998
Epoch 4: val_loss improved from 0.31199 to 0.29872, saving model to best_model.h5
624/624 [==============================] - 4s 7ms/step - loss: 0.2963 - accuracy: 0.8996 - val_loss: 0.2987 - val_accuracy: 0.9036
Epoch 5/10
617/624 [============================>.] - ETA: 0s - loss: 0.2639 - accur

In [15]:
model.load_weights('best_model.h5')

In [16]:
score = model.evaluate(X_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Test accuracy is ", accuracy)

Test accuracy is  91.3902223110199


### 5. Alphabet Recognition System

In [17]:
from keras.models import load_model

In [18]:
model = load_model('best_model.h5')

In [19]:
letters ={ 0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i', 9:'j', 10:'k', 11:'l', 
          12:'m', 13:'n', 14:'o', 15:'p', 16:'q', 17:'r', 18:'s', 19:'t', 20:'u', 21:'v', 22:'w', 
          23:'x', 24:'y', 25:'z', 26:''}

In [20]:
import numpy as np
 
blueLower = np.array([100,60,60])
blueUpper = np.array([140,255,255])

In [21]:
kernel = np.ones((5,5), np.uint8)

In [22]:
blackboard = np.zeros((480,640, 3), dtype=np.uint8)
alphabet = np.zeros((200,200,3), dtype=np.uint8)

In [23]:
from collections import deque
points = deque(maxlen = 512)

### 6. Open Web Camera and recogonise Alphabet

In [ ]:
import cv2 
cap = cv2.VideoCapture(0)
prediction = 26
while True:
    ret,frame=cap.read()
    frame = cv2.flip(frame, 1) 
    
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    blue = cv2.inRange(hsv, blueLower, blueUpper)
    blue = cv2.erode(blue, kernel)
    blue = cv2.morphologyEx(blue, cv2.MORPH_OPEN, kernel)
    blue = cv2.dilate(blue, kernel)
    cnts , _ = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
      
    center = None
    if len(cnts) > 0:
        cnt = sorted(cnts, key = cv2.contourArea, reverse=True)[0]
        ((x,y), radius) = cv2.minEnclosingCircle(cnt)
        cv2.circle(frame, (int(x), int(y),), int(radius), (125,344,278), 2)
        M = cv2.moments(cnt)
        center = (int(M['m10']/M['m00']), int(M['m01']/M['m00']))
        points.appendleft(center)
    elif len(cnts) == 0:
        if len(points) != 0:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur = cv2.medianBlur(blackboard_gray, 15)
            blur = cv2.GaussianBlur(blur, (5,5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            #cv2.imshow("Thresh", thresh)
            blackboard_cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
             
            if len(blackboard_cnts)>=1:
                cnt = sorted(blackboard_cnts, key=cv2.contourArea, reverse=True)[0]
                 
                if cv2.contourArea(cnt)>1000:
                    x,y,w,h = cv2.boundingRect(cnt)
                    alphabet = blackboard_gray[y-10:y+h+10,x-10:x+w+10]
                    try:
                        img = cv2.resize(alphabet, (28,28))
                    except cv2.error as e:
                        continue 
                    img = np.array(img)
                    img = img.astype('float32')/255
                    prediction = model.predict(img.reshape(1,28,28))[0]
                    prediction = np.argmax(prediction)
            points = deque(maxlen=512)
            blackboard = np.zeros((480,640, 3), dtype=np.uint8)
    for i in range(1, len(points)):
        if points[i-1] is None or points[i] is None:
            continue
        cv2.line(frame, points[i-1], points[i], (0,0,0), 2)
        cv2.line(blackboard, points[i-1], points[i], (255,255,255), 8)
    cv2.putText(frame, "Prediction is : " + str(letters[int(prediction)]), (20,400), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    
    cv2.imshow("Alphabet Recognition System",frame)
    
    if cv2.waitKey(1)==13:
        break
cap.release()
cv2.destroyAllWindows()